In [82]:
import numpy as np
from breizhcrops import BreizhCrops
import pandas as pd
import geopandas as gpd
import contextily as ctx

import torch
import torch.optim as optim
import torch.nn as nn
import torch.autograd as autograd
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset

import torchvision
import numpy as np
import math

from tqdm.notebook import tqdm

In [46]:
frh01 = BreizhCrops(region="frh01")
frh02 = BreizhCrops(region="frh02")
frh03 = BreizhCrops(region="frh03")
frh04 = BreizhCrops(region="frh04")

frh01.csv: 13.4MB [00:00, 41.8MB/s]                            
frh01.h5.tar.gz:   0%|          | 2.15M/1.64G [00:00<01:16, 21.5MB/s]

downloading breizhcrops_dataset/2017/L1C/frh01.h5.tar.gz


frh01.h5.tar.gz: 1.64GB [00:17, 91.8MB/s]                            


extracting breizhcrops_dataset/2017/L1C/frh01.h5.tar.gz to breizhcrops_dataset/2017/L1C/frh01.h5
removing breizhcrops_dataset/2017/L1C/frh01.h5.tar.gz
checking integrity by file size...
ok!


frh02.csv: 11.2MB [00:00, 48.7MB/s]                            
frh02.h5.tar.gz:   0%|          | 4.81M/1.45G [00:00<00:30, 48.0MB/s]

...

extracting breizhcrops_dataset/2017/L1C/frh02.h5.tar.gz to breizhcrops_dataset/2017/L1C/frh02.h5
removing breizhcrops_dataset/2017/L1C/frh02.h5.tar.gz
checking integrity by file size...
ok!


frh03.csv: 12.6MB [00:00, 41.5MB/s]                            
frh03.h5.tar.gz:   0%|          | 1.69M/1.60G [00:00<01:34, 16.9MB/s]

downloading breizhcrops_dataset/2017/L1C/frh03.h5.tar.gz


frh03.h5.tar.gz: 1.60GB [00:17, 91.2MB/s]                            


extracting breizhcrops_dataset/2017/L1C/frh03.h5.tar.gz to breizhcrops_dataset/2017/L1C/frh03.h5
removing breizhcrops_dataset/2017/L1C/frh03.h5.tar.gz
checking integrity by file size...
ok!


In [75]:
element_idx = np.arange(0, len(frh01))

In [76]:
np.random.seed(0)

In [77]:
train_len = int(0.7*len(element_idx))
val_len = int(0.15*len(element_idx))
test_len = int(0.15*len(element_idx))

In [78]:
np.random.shuffle(element_idx)

In [79]:
train_idx = element_idx[:train_len]
val_idx = element_idx[train_len:train_len+val_len]
test_idx = element_idx[train_len+val_len:]

In [90]:
class Brezz(Dataset):
    def __init__(self, brez_data, idxs, transform=None):
        self.brez_data = brez_data
        self.idxs = idxs
        self.transform = transform

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return self.brez_data[self.idxs[idx]]

In [91]:
train = Brezz(brez_data=frh01, idxs=train_idx)
val = Brezz(brez_data=frh01, idxs=val_idx)
test = Brezz(brez_data=frh01, idxs=test_idx)